# Pretrained model evaluation

This notebook evaluates models in the pretraining pool.

## Change working directory to project root

In [ ]:
import os
ROOT_DIRECTORIES = {'dogwood', 'tests'}
if set(os.listdir('.')).intersection(ROOT_DIRECTORIES) != ROOT_DIRECTORIES:
    os.chdir('../..')

## Exploration

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import smart_resize
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.applications.efficientnet import \
    preprocess_input as preprocess_input_effnet
from mlops.dataset.versioned_dataset import VersionedDataset
from mlops.model.versioned_model import VersionedModel
from dogwood.pretraining.pretraining_pool import PretrainingPool, \
    DATASET_MINI_IMAGENET

In [ ]:
pool = PretrainingPool()
print('Models:')
for model_path in pool.get_available_models():
    print(model_path)
print()
print('Datasets:')
for dataset_path in pool.get_available_datasets():
    print(dataset_path)

In [ ]:
imagenet_dataset = VersionedDataset(
    '/Users/leo/.dogwood/pretrained/datasets/imagenet-mini/v1')

In [ ]:
imagenet_dataset.X_train.shape

In [ ]:
imagenet_dataset.X_train.dtype

In [ ]:
plt.imshow(imagenet_dataset.X_train[0])

### VGG16

In [ ]:
X_train_resized = smart_resize(imagenet_dataset.X_train, (224, 224))

In [ ]:
X_train_resized.shape

In [ ]:
X_train_resized.dtype

In [ ]:
X_train_resized.max()

In [ ]:
plt.imshow(X_train_resized[0] / 255)

In [ ]:
imagenet_dataset.y_train.shape

In [ ]:
np.isclose(preprocess_input(X_train_resized[:2]), X_train_resized[:2])

In [ ]:
X_train_preprocessed = preprocess_input(X_train_resized)

In [ ]:
vgg = VersionedModel('/Users/leo/.dogwood/pretrained/models/VGG16/v1')

In [ ]:
vgg.model.compile(loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
vgg.model.evaluate(
    X_train_preprocessed,
    imagenet_dataset.y_train,
    batch_size=32)

### EfficientNetB7

In [ ]:
X_train_resized = smart_resize(imagenet_dataset.X_train[:1000], (600, 600))

In [ ]:
X_train_resized.shape

In [ ]:
X_train_resized.dtype

In [ ]:
X_train_resized.max()

In [ ]:
X_train_preprocessed = preprocess_input_effnet(X_train_resized)

In [ ]:
effnet = VersionedModel(
    '/Users/leo/.dogwood/pretrained/models/EfficientNetB7/v1')

In [ ]:
effnet.model.compile(
    loss='categorical_crossentropy',
    metrics=['accuracy', 'top_k_categorical_accuracy'])

In [ ]:
effnet.model.evaluate(
    X_train_preprocessed,
    imagenet_dataset.y_train[:1000])